In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
df_cross_1 = pd.read_csv('Data_2 (CNN + Little_Listing_Features + SVD).csv')

# Xgboost

In [26]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [24]:
X = df_cross_1.drop(columns = 'preference')
y = df_cross_1['preference']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [27]:
xgb1 = xgb.XGBRegressor()
parameters = {
              'objective':['reg:linear','reg:squarederror'],
              'learning_rate': [.03, 0.05], #so called `eta` value
              'max_depth': [5, 6, 7],
              'n_estimators': [10,20]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:  2.5min finished


[02:27:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=2, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=5,
             param_grid={'learning_rate': [0.03, 0.05], 'max_depth': [5, 6, 7],
                         'n_estimators': [10, 20],
    

In [28]:
print('best parameters: ', xgb_grid.best_params_)

best parameters:  {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 20, 'objective': 'reg:linear'}


In [30]:
y_train_pred = xgb_grid.predict(X_train)
y_pred = xgb_grid.predict(X_test)
mse_training = mean_squared_error(xgb_grid.predict(X_train), y_train)
mse_testing = mean_squared_error(xgb_grid.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.03580842196156337, testing mse: 0.03651029904889397
training rmse: 0.1892311337004653, testing rmse: 0.19107668368718872


# Random Forest

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [32]:
forest = RandomForestRegressor(random_state = 1, n_estimators = 10, min_samples_split = 2)
params = {
            'max_depth': [5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5]
        }

rf = GridSearchCV(forest, params, cv = 3, verbose = 1, n_jobs = -1)
rf.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 16.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=1,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-

In [33]:
print('best parameters: ', rf.best_params_)

best parameters:  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}


In [34]:
from sklearn.metrics import mean_squared_error as mse

y_train_pred = rf.predict(X_train)
y_pred = rf.predict(X_test)
mse_training = mse(rf.predict(X_train), y_train)
mse_testing = mse(rf.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.019589441483705474, testing mse: 0.021780560524517524
training rmse: 0.13996228593340948, testing rmse: 0.14758238554962283


# Linear Regression

In [35]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [36]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
import numpy as np

def mse(y_predicted, y_true):
  return np.mean(np.square(y_predicted - y_true))
mse_training = mse(lr.predict(X_train), y_train)
mse_testing = mse(lr.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.023163369335312992, testing mse: 0.02327799105419951
training rmse: 0.1521951685675764, testing rmse: 0.1525712654932098
